## 1、Transformer 简介：

---

**Transformer** 是一种基于Self-Attention的模型，是**seq2seq模型**，主要处理NLP。它由 **编码器（Encoder）** 和 **解码器（Decoder）** 两部分组成。

- **编码器**：通过多层堆叠的Self-Attention和前馈神经网络（Feed-Forward Network）来处理，对**词向量融合语法特征、语义特征，得到新的词向量**。
- **解码器**：通过Mask-Self-Attention、融合**编码器的输出KV和解码器输入Q**的编码器和解码器注意力层和前馈网络来处理。

##### 1. **编码器部分**：
- **输入嵌入与位置编码**：首先，输入的每个词通过嵌入（Embedding）映射为向量，并加上 **位置编码**（Positional Encoding），以保留词语的顺序信息。
  
- **自注意力机制**：每个词向量的表示会通过与其他所有词的关系来调整，计算注意力权重，融合多方面的特征信息。具体计算包括 **查询（Q）**、**键（K）** 和 **值（V）** 的关系。

- **多头注意力**：堆叠多次Self-Attention，使用多个注意力头并行处理，从不同的角度捕捉更多的信息，拼接后通过线性变换生成最终表示。

- **前馈神经网络**：通过一个前馈神经网络对每个位置的输出进行处理，并采用 **残差连接+归一化**来加速训练。

##### 2. **解码器部分**：
解码器的主要任务是根据编码器的输出生成目标序列，工作流程如下：

- **目标序列嵌入与位置编码**：解码器的输入也是经过嵌入和位置编码处理的目标序列。

- **自注意力机制（Masked）**：解码器的自注意力是 **mask** 的，这是为了保证训练阶段和测试阶段是相同的，不能使模型看到未来的信息。

- **编码器-解码器注意力**：这里的QKV不是同源的，有点类似交叉注意力，Q是解码器的输入端，KV是编码器的输出端。

- **前馈神经网络与残差连接**：同编码器一样，解码器中的每个位置也会通过前馈神经网络进行处理，并使用残差连接。


- **线性变换与输出**：解码器最后的输出通过一个线性层（通常是一个全连接层）映射到目标词汇表大小，生成最终的预测输出。

## 2、Transformer 为何使用多头注意力机制？为什么不使用一个头？

---

Transformer 中使用多头注意力机制（Multi-Head Attention），核心目的是提升模型的表示能力。相比单一注意力头，多头结构能够从多个子空间并行建模不同的注意力关系，捕捉更丰富的语义信息。

### 🎯 表达多样性（Representation Diversity）

单个注意力头只能在一个空间中计算 token 之间的关系，表达能力有限；而多头注意力通过多个线性变换，将 Q、K、V 投影到不同的子空间：

$$
\text{head}_i = \text{Attention}(QW_i^Q, KW_i^K, VW_i^V)
$$

每个 Head 学习到的关注模式是不同的：
- 有的 Head 关注短程依赖（如词法搭配）
- 有的 Head 关注长程依赖（如主谓一致、跨句结构）
- 有的 Head 可能自然学习到句法或语义角色

这些不同关注角度最终被拼接再整合：

$$
\text{MultiHead}(Q,K,V) = \text{Concat}(\text{head}_1,\dots,\text{head}_h)W^O
$$

---

### ✅ 总结一句话：

Transformer 使用 **多头注意力机制** 主要是为了使模型捕捉到更丰富的**特征信息**。使用一个头只能将词向量转换到一个空间中进行计算，限制了信息捕捉的多样性。而 **多头机制** 通过多个头并行计算**不同子空间的注意力**，能够同时学习到词语与词语之间的多种关系，提升了模型的表达和泛化能力。此外，多头机制也利用了 **并行计算** 的优势，提升了效率。


